In [4]:
import torch
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained('BM-K/KoSimCSE-roberta')
tokenizer = AutoTokenizer.from_pretrained('BM-K/KoSimCSE-roberta')

In [5]:
def cal_score(a, b):
    if len(a.shape) == 1: a = a.unsqueeze(0)
    if len(b.shape) == 1: b = b.unsqueeze(0)

    a_norm = a / a.norm(dim=1)[:, None]
    b_norm = b / b.norm(dim=1)[:, None]
    return torch.mm(a_norm, b_norm.transpose(0, 1)) * 100

In [6]:
sentences = ['치타가 들판을 가로 질러 먹이를 쫓는다.',
             '치타 한 마리가 먹이 뒤에서 달리고 있다.',
             '원숭이 한 마리가 드럼을 연주한다.']

inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
embeddings, _ = model(**inputs, return_dict=False)

score01 = cal_score(embeddings[0][0], embeddings[1][0])
score02 = cal_score(embeddings[0][0], embeddings[2][0])

데이터셋: 토스 QnA 활용 (https://support.toss.im/faq?category=0&page=1)

In [7]:
qs = ["연말정산 월세 세액 공제 위해서 송금확인증 발급이 필요해요.",
      "토스증권 수수료와 세금이 궁금해요!",
      "토스프라임은 어떤 서비스인가요?",
      "토스뱅크 계좌의 송금한도는 어떻게 되나요?",
      "해외 송금도 가능한가요?",
      "토스에 연결된 계좌의 송금 한도가 궁금해요.",
      "토스 송금은 어떤 시스템을 통해 이뤄지나요?",
      "모임카드로 사용한 금액에 대해 연말정산 받을 수 있나요?",
      "토스뱅크 체크카드, 모임카드는 연말정산 시 어떻게 적용하나요?",
      "토스뱅크 대출은 연말정산 시 소득공제 가능한가요?",
      "토스유스카드 연말정산, 소득공제 어떻게 하나요?",
      "연말정산 월세 세액 공제 위해서 송금확인증 발급이 필요해요.",
     ]

answers = ["송금확인증은 토스 앱으로 팩스, 이메일, 다운로드 발급이 가능해요. 아래의 경로에 따라 발급해주세요. \n - '전체' → '고객센터' → '각종 증명서 발급' → '토스뱅크 증명서' → '송금확인증' \n직접 송금한 내역을 선택해서 발급해주세요.",
          """토스증권에서 거래시 발생하는 수수료는 아래와 같습니다. 



국내 주식 수수료

주식 구매 시: 수수료 0.015%
주식 판매 시: 수수료 0.015% + 제세금 0.18%

해외 주식 수수료

주식 구매 시: 수수료 0.1%
주식 판매 시: 0.1% + 유관 수수료 (SEC Fee: 판매 금액의 0.0008% (최소 0.01 달러) / TAF: 판매 수량 당 0.000166달러 (최소 0.01 달러, 최대 8.3달러))""",
           """토스에서 아래의 혜택을 받아볼 수 있는 매월 5,900원의 정기 구독형 프리미엄 유료 서비스예요.

토스프라임 혜택을 소개해 드릴게요. 

 토스증권 국내 주식 수수료 캐시백 (0.015%)
  - 국내 주식 수수료 거래액 누적 1억 원까지
 토스페이를 통한 가맹점 결제 시 토스포인트 적립 (최대 12,000원) 
- 결제 금액 누적 60만원까지 적립 받아볼 수 있어요.
* [20만원 까지 4%적립 / 이후 60만원 까지 1% 적립]
* [후불결제건도 동일하게 적립]
토스 포인트 무료 출금권 1회 지급
-토스포인트를 1회 무료로 출금할 수 있어요.
(무료 출금권을 사용하지 않았다면, 다음 달로 이월되어요.)


자세한 내용은 토스 앱 ‘전체’ → ‘토스프라임’ 에서 확인 가능해요.""",
           """[모바일앱 송금한도]

한도제한계좌 해제 고객 : 1일 5천만 원, 1회 1천만 원
추가인증 이용 고객 : 1일 5억 원, 1회 1억 원
임시증액 신청 고객 : 1일 20억 원, 1회 10억 원
토스뱅크 채우기 : 전 금융기관 합산 1일 최대 1천만 원 
추가인증 등록과 임시증액은 각각 아래의 경로를 통해 진행할 수 있어요. 

추가인증(OTP)등록 : 전체 → 설정 → 송금 → 토스뱅크 인증수단 관리 
임시증액 : 홈 → 토스뱅크통장 → 톱니바퀴 모양의 버튼(관리) → 내 송금 한도 → 임시 증액하기


※ 한도계좌의 경우 1일/1회 : 2백만 원 까지 가능해요.

※ 미성년자 한도계좌의 경우  1일/1회 최대 1백만 원 까지 가능해요.

※ 외국인 한도계좌의 경우  1일/1회 최대 30만 원 까지 가능해요.

※ 입금하는 곳이 계좌가 아닌 연락처일 경우 토스의 송금한도에 따라 1회 200만 원/1일 1000만 원까지 가능한 점 참고 부탁드려요. 

※ 한도계좌 해제 방법은 이 페이지에서 자세하게 확인할 수 있어요. (바로가기)""",
           """안타깝게도 토스를 통한 해외 송금은 이용이 어려워요.

일부 외국계 은행 (홍콩상하이은행, 중국공상은행, 인도네시아느가라은행 등)이라면 대한민국에 있는 지점에서 개설하신 계좌일 경우에 한해 국내 송금만 이용이 가능해요.""",
           """만 19세 이상 성인의 경우
1회 200만 원 / 1일 1,000만 원


만 19세 미만 미성년자의 경우
1회 50만 원 / 1일 100만 원 / 1개월 200만 원


만 19세 미만 미성년자 토스머니 입금한도
연락처 송금 : 1회 50만 원
가상 계좌번호 : 1회 20만 원 / 월 100만 원
토스 유스 카드를 사용 중이라면 토스머니 충전도 한도에 포함돼요.


위의 한도에서 늘리거나 줄이는 것은 불가능해요.
토스는 금융사의 한도를 우선해서 적용하고 있어요. 한도를 높이길 원한다면 금융사로 문의해 주세요.



* 토스뱅크 계좌는 별도의 한도가 적용되어요.
    토스뱅크 계좌 한도 보러가기

* 일부 은행은 한도가 정해져 있어요.

메리츠증권 : 일 150만 원
키움증권 : 일 200만 원 / 월 500만 원
신한금융투자, 하나은행, 경남은행 : 일 200만 원
토스에서 개설한 비대면계좌 (신한금융투자, 하나은행, 수협) : 일 200만 원
케이뱅크 : 토스머니 충전 일 10회 / 월 30회""",
           """토스는 CMS(Cash Management Service)를 사용하고 있어요.

* CMS: 은행이 운영하는 자금관리 서비스. 매달 자동 이체되는 기부금, 정기예금, 공과금 등에 주로 사용되는 출금 시스템""",
           """모임카드를 사용한 연말정산 반영은 모임카드를 발급한 본인만 연말정산 받을 수 있어요.
즉, 모임장과 공동명의자 3명이 모임카드를 각각 발급받고 각자 사용하였다면 각자에게 반영이 돼요.""",
           """토스뱅크 체크카드, 모임카드 사용 내역은 자동으로 홈택스에 반영되기에 따로 연말정산을 신청하지 않아도 괜찮아요.

다만, 월세 세액공제를 위해선 월세를 송금한 송금확인증 발급이 필요해요. 토스뱅크 통장을 통해 월세를 송금했다면 아래의 경로로 송금확인증 발급을 해주세요.
●토스 앱 → '전체' → '토스뱅크' → '토스뱅크 통장' → 오른쪽 상단 '관리' → 증명서 발급하기 → 송금확인증

※ 2022년 금융소득원천징수 영수증은 2023년 3월 1일 이후부터 발급할 수 있어요.""",
           """토스뱅크는 신용대출만 진행하고 있고 신용대출은 소득공제 대상이 아니에요 

※ 연말정산 공제 가능 대출 항목은 주택 관련 대출만 가능하며 '주택담보대출, 전세 및 월세 자금 대출'인 경우 근로소득자, 세대주, 주택 규모 등 기본적인 조건이 충족된다면 소득에서 일정 부분 공제를 해드리고 있어요.""",
           """2023년도 유스카드 연말정산 소득공제 신청을 2023년 안에 신청하지 못했다면 고객센터를 통해 자료를 요청할 수 있어요.

주말과 공휴일을 제외한 영업일 기준 3~4일 안에 알려주신 이메일 또는 팩스로 보내드릴게요.
발송되면 토스에 등록된 휴대폰 번호로 문자를 보내드릴게요.


[소득공제 자료 신청 방법]

① 자녀가 만 14세 미만인 경우

자녀 또는 가입 당시 인증해주신 법정대리인께서 고객센터로 자료 요청


② 자녀가 만 14세 이상인 경우

자녀 본인께서 고객센터로 자료 요청


[국세청 홈택스에서 자녀를 부양가족으로 등록하는 방법]

등록 경로 : '국세청 홈택스' → 로그인(토스인증서 가능) → '연말정산 간소화' → '부양가족 자료제공 동의 현황 조회' → '자료제공 동의 신청' → '미성년자 자녀 신청' → 정보 입력 후 '신청하기'
확인 경로 : '국세청 홈택스' → 로그인(토스인증서 가능) → '연말정산 간소화' → '소득공제 자료 조회' → '직불카드 등'
토스에 등록된 법정대리인이 아니어도 소득공제를 신청할 수 있어요.
부양가족 등록 관련 도움이 필요하신 경우, 국세청 고객센터(국번 없이 126)로 문의해주세요.""",
           """송금확인증은 토스 앱으로 팩스, 이메일, 다운로드 발급이 가능해요. 아래의 경로에 따라 발급해주세요. 

'전체' → '고객센터' → '각종 증명서 발급' → '토스뱅크 증명서' → '송금확인증'
직접 송금한 내역을 선택해서 발급해주세요.
[토스뱅크 송금확인증 바로가기]


혹시 송금하신 내역이 보이지않나요?
토스 뱅크 계좌에서 송금하신 내역이 아니라면 확인이 어려워요.
토스에 연결되어있는 타 금융계좌를 통한 송금 건은 아니었는지 확인해주세요. 

[토스 송금확인증 바로가기]""",
           
          ]

In [8]:
qs_tokens = tokenizer(qs, padding=True, truncation=True, return_tensors="pt")
qs_embeddings, _ = model(**qs_tokens, return_dict=False)
qs_embeddings.shape

torch.Size([12, 20, 768])

In [9]:
query = "토스 송금 한도에 대해 궁금해요"

query_token = tokenizer(query, padding=True, truncation=True, return_tensors="pt")
query_embedding, _ = model(**query_token, return_dict=False)

results = cal_score(query_embedding[0][0], qs_embeddings[:, 0, :])
cands = results.topk(k=3).indices.tolist()[0]
scores = results.topk(k=3).values.tolist()[0]

print(f"질문: {query} \n")
for idx, score in zip(cands, scores):
    print(f"후보질문: {qs[idx]}")
    print(f"Index: {idx} Scores: {score}")
    print(answers[idx])
    print("="*100)

질문: 토스 송금 한도에 대해 궁금해요 

후보질문: 토스에 연결된 계좌의 송금 한도가 궁금해요.
Index: 5 Scores: 94.55921936035156
만 19세 이상 성인의 경우
1회 200만 원 / 1일 1,000만 원


만 19세 미만 미성년자의 경우
1회 50만 원 / 1일 100만 원 / 1개월 200만 원


만 19세 미만 미성년자 토스머니 입금한도
연락처 송금 : 1회 50만 원
가상 계좌번호 : 1회 20만 원 / 월 100만 원
토스 유스 카드를 사용 중이라면 토스머니 충전도 한도에 포함돼요.


위의 한도에서 늘리거나 줄이는 것은 불가능해요.
토스는 금융사의 한도를 우선해서 적용하고 있어요. 한도를 높이길 원한다면 금융사로 문의해 주세요.



* 토스뱅크 계좌는 별도의 한도가 적용되어요.
    토스뱅크 계좌 한도 보러가기

* 일부 은행은 한도가 정해져 있어요.

메리츠증권 : 일 150만 원
키움증권 : 일 200만 원 / 월 500만 원
신한금융투자, 하나은행, 경남은행 : 일 200만 원
토스에서 개설한 비대면계좌 (신한금융투자, 하나은행, 수협) : 일 200만 원
케이뱅크 : 토스머니 충전 일 10회 / 월 30회
후보질문: 토스뱅크 계좌의 송금한도는 어떻게 되나요?
Index: 3 Scores: 82.73033905029297
[모바일앱 송금한도]

한도제한계좌 해제 고객 : 1일 5천만 원, 1회 1천만 원
추가인증 이용 고객 : 1일 5억 원, 1회 1억 원
임시증액 신청 고객 : 1일 20억 원, 1회 10억 원
토스뱅크 채우기 : 전 금융기관 합산 1일 최대 1천만 원 
추가인증 등록과 임시증액은 각각 아래의 경로를 통해 진행할 수 있어요. 

추가인증(OTP)등록 : 전체 → 설정 → 송금 → 토스뱅크 인증수단 관리 
임시증액 : 홈 → 토스뱅크통장 → 톱니바퀴 모양의 버튼(관리) → 내 송금 한도 → 임시 증액하기


※ 한도계좌의 경우 1일/1회 : 2백만 원 까지 가능해요.

※ 미성년자 한도계좌의 경

In [10]:
query = "월세 연말정산은 어떻게 신청하나요?"

query_token = tokenizer(query, padding=True, truncation=True, return_tensors="pt")
query_embedding, _ = model(**query_token, return_dict=False)

results = cal_score(query_embedding[0][0], qs_embeddings[:, 0, :])
cands = results.topk(k=3).indices.tolist()[0]
scores = results.topk(k=3).values.tolist()[0]

print(f"질문: {query} \n")
for idx, score in zip(cands, scores):
    print(f"후보질문: {qs[idx]}")
    print(f"Index: {idx} Scores: {score}")
    print(answers[idx])
    print("="*100)

질문: 월세 연말정산은 어떻게 신청하나요? 

후보질문: 토스유스카드 연말정산, 소득공제 어떻게 하나요?
Index: 10 Scores: 76.88180541992188
2023년도 유스카드 연말정산 소득공제 신청을 2023년 안에 신청하지 못했다면 고객센터를 통해 자료를 요청할 수 있어요.

주말과 공휴일을 제외한 영업일 기준 3~4일 안에 알려주신 이메일 또는 팩스로 보내드릴게요.
발송되면 토스에 등록된 휴대폰 번호로 문자를 보내드릴게요.


[소득공제 자료 신청 방법]

① 자녀가 만 14세 미만인 경우

자녀 또는 가입 당시 인증해주신 법정대리인께서 고객센터로 자료 요청


② 자녀가 만 14세 이상인 경우

자녀 본인께서 고객센터로 자료 요청


[국세청 홈택스에서 자녀를 부양가족으로 등록하는 방법]

등록 경로 : '국세청 홈택스' → 로그인(토스인증서 가능) → '연말정산 간소화' → '부양가족 자료제공 동의 현황 조회' → '자료제공 동의 신청' → '미성년자 자녀 신청' → 정보 입력 후 '신청하기'
확인 경로 : '국세청 홈택스' → 로그인(토스인증서 가능) → '연말정산 간소화' → '소득공제 자료 조회' → '직불카드 등'
토스에 등록된 법정대리인이 아니어도 소득공제를 신청할 수 있어요.
부양가족 등록 관련 도움이 필요하신 경우, 국세청 고객센터(국번 없이 126)로 문의해주세요.
후보질문: 토스뱅크 체크카드, 모임카드는 연말정산 시 어떻게 적용하나요?
Index: 8 Scores: 64.82209014892578
토스뱅크 체크카드, 모임카드 사용 내역은 자동으로 홈택스에 반영되기에 따로 연말정산을 신청하지 않아도 괜찮아요.

다만, 월세 세액공제를 위해선 월세를 송금한 송금확인증 발급이 필요해요. 토스뱅크 통장을 통해 월세를 송금했다면 아래의 경로로 송금확인증 발급을 해주세요.
●토스 앱 → '전체' → '토스뱅크' → '토스뱅크 통장' → 오른쪽 상단 '관리' → 증명서 발급하기 → 송금확인증

※ 2022년 금융소득원천징수 영수증은 

In [23]:
query = "해외로 돈 보내는 방법 알려줘"

query_token = tokenizer(query, padding=True, truncation=True, return_tensors="pt")
query_embedding, _ = model(**query_token, return_dict=False)

results = cal_score(query_embedding[0][0], qs_embeddings[:, 0, :])
cands = results.topk(k=3).indices.tolist()[0]
scores = results.topk(k=3).values.tolist()[0]

print(f"질문: {query} \n")
for idx, score in zip(cands, scores):
    print(f"후보질문: {qs[idx]}")
    print(f"Index: {idx} Scores: {score}")
    print(answers[idx])
    print("="*100)

질문: 해외로 돈 보내는 방법 알려줘 

후보질문: 해외 송금도 가능한가요?
Index: 4 Scores: 71.14073181152344
안타깝게도 토스를 통한 해외 송금은 이용이 어려워요.

일부 외국계 은행 (홍콩상하이은행, 중국공상은행, 인도네시아느가라은행 등)이라면 대한민국에 있는 지점에서 개설하신 계좌일 경우에 한해 국내 송금만 이용이 가능해요.
후보질문: 토스 송금은 어떤 시스템을 통해 이뤄지나요?
Index: 6 Scores: 49.91534423828125
토스는 CMS(Cash Management Service)를 사용하고 있어요.

* CMS: 은행이 운영하는 자금관리 서비스. 매달 자동 이체되는 기부금, 정기예금, 공과금 등에 주로 사용되는 출금 시스템
후보질문: 토스뱅크 계좌의 송금한도는 어떻게 되나요?
Index: 3 Scores: 40.28041458129883
[모바일앱 송금한도]

한도제한계좌 해제 고객 : 1일 5천만 원, 1회 1천만 원
추가인증 이용 고객 : 1일 5억 원, 1회 1억 원
임시증액 신청 고객 : 1일 20억 원, 1회 10억 원
토스뱅크 채우기 : 전 금융기관 합산 1일 최대 1천만 원 
추가인증 등록과 임시증액은 각각 아래의 경로를 통해 진행할 수 있어요. 

추가인증(OTP)등록 : 전체 → 설정 → 송금 → 토스뱅크 인증수단 관리 
임시증액 : 홈 → 토스뱅크통장 → 톱니바퀴 모양의 버튼(관리) → 내 송금 한도 → 임시 증액하기


※ 한도계좌의 경우 1일/1회 : 2백만 원 까지 가능해요.

※ 미성년자 한도계좌의 경우  1일/1회 최대 1백만 원 까지 가능해요.

※ 외국인 한도계좌의 경우  1일/1회 최대 30만 원 까지 가능해요.

※ 입금하는 곳이 계좌가 아닌 연락처일 경우 토스의 송금한도에 따라 1회 200만 원/1일 1000만 원까지 가능한 점 참고 부탁드려요. 

※ 한도계좌 해제 방법은 이 페이지에서 자세하게 확인할 수 있어요. (바로가기)


llamaindex

In [17]:
from llama_index import Document
from llama_index.embeddings import HuggingFaceEmbedding

documents = [Document(text=t) for t in answers]
embed_model = HuggingFaceEmbedding(model=model, tokenizer=tokenizer)

In [18]:
embeddings = embed_model.get_text_embedding("Hello World!")
print(len(embeddings))
print(embeddings[:5])

768
[-0.0005478347884491086, 0.015290956012904644, 0.014890926890075207, -0.01885056495666504, 0.03864099085330963]
